# Code for AISTATS submission (Reconstructing Test Labels from Noisy Loss Functions)
## Label inference with Itakura-Saito loss on titanic

### Experiment setup

In [1]:
import os
import sys
import json
import pandas as pd
import numpy as np
from collections import Counter
import math
import time
import warnings
warnings.filterwarnings('ignore')

np.random.seed(12345)

### Titanic data

In [2]:
titanic = pd.read_csv('data/titanic.csv')
titanic = titanic.sample(frac=1)
print(titanic.shape) 
print(Counter(titanic['class']))
titanic.head()

(2201, 4)
Counter({-1: 1490, 1: 711})


Class    Age    Sex  class
1808  0.9650 -0.228  0.521     -1
2040  0.0214 -0.228  0.521     -1
1212  0.9650 -0.228  0.521     -1
63    0.0214 -0.228  0.521     -1
1805  0.0214 -0.228  0.521     -1

In [3]:
def titanic_to_int(x):
    return x if x == 1 else 0
titanic_label = titanic['class'].apply(titanic_to_int) 
Counter(titanic_label)

Counter({0: 1490, 1: 711})

## utility functions in the experiments

In [4]:
# Method to compute binary log-loss
def itakura_loss(labeling = [], pred=[], tau=0):
    """
    labeling: ground truth labels
    pred: prediction vector
    Return itakura-saito score
    """
    if not pred:
        raise ValueError("No prediction vector passed")
    N = len(labeling)
    assert len(pred)==N
    loss = 0
    for i in range(N):
        theta0 = sys.float_info.min if 1 - pred[i] < sys.float_info.min else 1 - pred[i]
        loss += labeling[i] * (1/(1-theta0) + math.log(1-theta0) - 1)
        loss += (1 - labeling[i]) * (1/theta0 + math.log(theta0) - 1)
        
    return loss/N + tau * np.random.rand()

In [5]:
# function to generate the predictions
def generate_prediction(N, tau_max):
    """
    N: number of data points
    tau_max: maximum noise
    Return prediction vector
    """
    prediction = np.zeros(N)
    for i in range(N):
        n = i + 1
        exponent = 2 ** n * N * tau_max
        prediction[i] = 1 / (1 + 3 ** (-exponent))
    return list(prediction)

In [6]:
# generator for K-nary labelings
def gen_K_ary(nbits, K=2):
    """
    nbits: number of bits in the K-nary vector
    K: K in K-nary, e.g. K=2 is binary, K=3 is ternary
    Returns a generator for all K-nary labelings of length nbits
    """
    for i in range(K**nbits):
        bin_str = str(np.base_repr(i, K))
        l = len(bin_str)
        bin_arr = np.zeros(nbits)
        for j in range(l):
            bin_arr[nbits-j-1] = float(bin_str[l-j-1])
        yield [int(x) for x in bin_arr]

In [7]:
# the main label inference algorithm
def main_algorithm(noisy_loss, N, K, prediction):
    """
    noisy_loss: observed loss score with noise
    N: number of data points
    K: number of classes
    prediction: 2-d numpy array, where prediction[i,k] represents the prediction for data i in class k
    Returns the inferred labeling
    """
    label_infer = []
    diff_min = np.inf
    lab_generator = gen_K_ary(N, K)
    
    while True:
        try:
            label_cur = next(lab_generator) 
            loss_cur = itakura_loss(label_cur, prediction, 0)
            diff_cur = np.abs(loss_cur - noisy_loss)
            # print(loss_cur, diff_cur)
            if diff_cur < diff_min:
                label_infer = label_cur.copy()
                diff_min = diff_cur
        except:
            break
    return label_infer

In [8]:
# the main function for simulation
def main_sim(true_labeling, N, K, tau_max, nsims=1000):
    """
    true_labeling: ground truth labeling with length at least N
    N: number of data points
    K: number of classes
    tau_max: maximum noise
    nsims: number of simulations
    Returns the average accuracy and the average loss score
    """
    acc = 0
    mean_loss = 0
    for ns in range(nsims):
        labeling = list(np.random.choice(true_labeling, N))
        prediction = generate_prediction(N, tau_max)
        noisy_loss = itakura_loss(labeling, prediction, tau_max)
        mean_loss += noisy_loss
        labeling_infer = main_algorithm(noisy_loss, N, K, prediction)
        all_same = (sum([x == y for x, y in zip(labeling_infer, labeling)]) == N)
        acc += all_same
    return acc / nsims, mean_loss / nsims

## label inference experiments

In [9]:
# logloss experiment 
nsims = 1000
result_itakura = []
K = 2

for tau_max in [1e-4, 1e-2, 1e-1, 1]:
    for N in range(1, 21):
        try:
            start = time.time()
            acc, mean_loss = main_sim(titanic_label, N, K, tau_max, nsims)
            seconds = time.time() - start
            result_itakura.append((K, tau_max, N, acc, mean_loss, seconds))
            print(K, tau_max, N, acc, mean_loss, seconds)
            if acc < 0.5: break
        except:
            break

2 0.0001 1 1.0 0.3069385703373167 0.10676980018615723
2 0.0001 2 1.0 0.30702512023670436 0.15200090408325195
2 0.0001 3 1.0 0.3071585491169907 0.25198912620544434
2 0.0001 4 1.0 0.3075295757558549 0.4715280532836914
2 0.0001 5 1.0 0.30805546080005985 0.9757139682769775
2 0.0001 6 1.0 0.30959654481922616 2.0681090354919434
2 0.0001 7 1.0 0.3122342200227298 4.735018968582153
2 0.0001 8 1.0 0.32008526744253024 9.968048095703125
2 0.0001 9 1.0 0.3406107214262265 21.6848247051239
2 0.0001 10 1.0 0.42728099622723553 47.55908989906311
2 0.0001 11 1.0 0.9905604450162434 104.01617288589478
2 0.0001 12 1.0 13.2891575652098 223.657949924469
2 0.0001 13 1.0 6092.514486941341 470.33163595199585
2 0.0001 14 1.0 4326244566.291707 1020.7579307556152
2 0.0001 15 0.315 inf 2142.216824054718
2 0.01 1 1.0 0.31610929796234616 0.09806704521179199
2 0.01 2 1.0 0.3249985834890613 0.15056276321411133
2 0.01 3 1.0 0.3486218044762942 0.2558012008666992
2 0.01 4 1.0 0.43799222298810986 0.496690034866333
2 0.01 5 

In [11]:
df_itakura = pd.DataFrame(result_itakura)
df_itakura.columns = ['K', 'tau_max', 'N', 'acc', 'mean_loss', 'seconds_1k']
df_itakura['ms'] = df_itakura['seconds_1k'] # since 1000 repetitions
df_itakura.to_csv('itakura_binary.csv', index=False)
print(df_itakura.shape)
df_itakura.head(10)

(35, 7)


K  tau_max   N  acc  mean_loss  seconds_1k         ms
0  2   0.0001   1  1.0   0.306939    0.106770   0.106770
1  2   0.0001   2  1.0   0.307025    0.152001   0.152001
2  2   0.0001   3  1.0   0.307159    0.251989   0.251989
3  2   0.0001   4  1.0   0.307530    0.471528   0.471528
4  2   0.0001   5  1.0   0.308055    0.975714   0.975714
5  2   0.0001   6  1.0   0.309597    2.068109   2.068109
6  2   0.0001   7  1.0   0.312234    4.735019   4.735019
7  2   0.0001   8  1.0   0.320085    9.968048   9.968048
8  2   0.0001   9  1.0   0.340611   21.684825  21.684825
9  2   0.0001  10  1.0   0.427281   47.559090  47.559090

# multiquery experiments

In [9]:
# the main function for simulation
def main_sim_multi(true_labeling, K, tau_max, shard_size, max_N_feasible):
    """
    true_labeling: ground truth labeling
    K: number of classes
    tau_max: maximum noise
    loss_type: either logloss or sigmoidloss
    shard_size: number of data points in each shard
    max_N_feasible: largest N such that the label inference algorithm is feasible
    Returns the average accuracy and the number of shards
    """
    acc = 0
    mean_loss = 0
    n_total = len(true_labeling)  
    n_shard = int(np.ceil(n_total / shard_size))
    for j in range(n_shard):
        if j < n_shard - 1:
            cur_shard_size = shard_size
            labeling = true_labeling[j*shard_size : (j+1)*shard_size]
        else:
            cur_shard_size = n_total - j*shard_size
            labeling = true_labeling[j*shard_size : n_total]
        N = int(min(max_N_feasible, cur_shard_size))
        labeling_cur = labeling[:N]
        tau = 2 * np.exp(1) * tau_max / (np.exp(1) - 1)
        prediction = generate_prediction(N, tau)
        noisy_loss = itakura_loss(labeling_cur, prediction, tau_max)
        labeling_infer = main_algorithm(noisy_loss, N, K, prediction)
        acc += sum([x == y for x, y in zip(labeling_infer, labeling_cur)])
    return acc / n_total, n_shard

In [10]:
loss_type = 'itakura'
itakura_multi = []

# read in the max_N_feasible for each K and tau_max setting from previous experimental results
setting_itakura = pd.read_csv('itakura_binary.csv')
titanic_label = titanic_label.tolist() 
df = setting_itakura.dropna().groupby(['K','tau_max'])['N'].max()
df = df.to_frame()
df = df.reset_index()
df.head()

K  tau_max   N
0  2   0.0001  15
1  2   0.0100   9
2  2   0.1000   6
3  2   1.0000   5

In [11]:
prev_acc = 0
for _, rows in df.iterrows():
    K, tau_max, max_N_feasible = int(rows[0]), rows[1], int(rows[2])
    shard_size_vec = [1,2,5,11,22,55,110,220,550,1100,2201]
    shard_size_vec = list(reversed(shard_size_vec))
    
    n_total = len(titanic_label)
    for i, shard_size in enumerate(shard_size_vec):
        start = time.time()
        acc, n_shards = main_sim_multi(titanic_label, K, tau_max, shard_size, max_N_feasible)
        if i > 0:
            acc = max(acc, prev_acc)
        seconds = time.time() - start
        itakura_multi.append((K, tau_max, acc, n_shards, seconds))
        print(K, tau_max, acc, n_shards, seconds)
        prev_acc = acc

2 0.0001 0.004543389368468878 1 1.9880619049072266
2 0.0001 0.009086778736937756 3 3.9807169437408447
2 0.0001 0.014538845979100408 5 8.4356849193573
2 0.0001 0.04179918218991367 11 20.15445899963379
2 0.0001 0.08405270331667423 21 39.92971706390381
2 0.0001 0.1858246251703771 41 79.75198483467102
2 0.0001 0.4606996819627442 101 200.893963098526
2 0.0001 1.0 201 21.05982804298401
2 0.0001 1.0 441 0.41344308853149414
2 0.0001 1.0 1101 0.1012263298034668
2 0.0001 1.0 2201 0.09865713119506836
2 0.01 0.0027260336210813267 1 0.01929616928100586
2 0.01 0.005452067242162653 3 0.03959774971008301
2 0.01 0.011812812358019082 5 0.07677078247070312
2 0.01 0.02407996365288505 11 0.19101786613464355
2 0.01 0.055883689232167195 21 0.3770310878753662
2 0.01 0.11313039527487506 41 0.7483861446380615
2 0.01 0.2780554293502953 101 1.9230568408966064
2 0.01 0.5601999091322126 201 3.913620948791504
2 0.01 1.0 441 0.3984189033508301
2 0.01 1.0 1101 0.09581804275512695
2 0.01 1.0 2201 0.09152793884277344
2 

In [12]:
itakura_multi = pd.DataFrame(itakura_multi)
itakura_multi.columns = ['K', 'tau_max', 'acc', 'n_shards', 'seconds']
itakura_multi.to_csv('itakura_multiquery_titanic.csv', index=False)
print(itakura_multi.shape)
itakura_multi.head(10)

(44, 5)


K  tau_max       acc  n_shards     seconds
0  2   0.0001  0.004543         1    1.988062
1  2   0.0001  0.009087         3    3.980717
2  2   0.0001  0.014539         5    8.435685
3  2   0.0001  0.041799        11   20.154459
4  2   0.0001  0.084053        21   39.929717
5  2   0.0001  0.185825        41   79.751985
6  2   0.0001  0.460700       101  200.893963
7  2   0.0001  1.000000       201   21.059828
8  2   0.0001  1.000000       441    0.413443
9  2   0.0001  1.000000      1101    0.101226